**Download required packages**


In [2]:
#%pip install --upgrade pip
#%pip install -U PyMuPDF
%pip install PyMuPDF-<...>.whl
#%pip install fitz
# %pip install datasets==1.2.1
# %pip install rouge_score

Note: you may need to restart the kernel to use updated packages.


Access is denied.


**Importing the libraries required**

In [ ]:
# for Pdf file
import PYMuPDF

# For wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Models for text summarization
from transformers import LEDForConditionalGeneration, LEDTokenizer,AutoConfig


# To ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Import Independenties.
from pathlib import Path
import torch
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


Import Model And It's Tokenizer And Config files

In [ ]:

model_path = f'E:/Model Matiral/the model'
tokenizer = LEDTokenizer.from_pretrained(model_path)
model = LEDForConditionalGeneration.from_pretrained(model_path)

config = AutoConfig.from_pretrained(model_path)

Load the pdf file And Print number of pages and metadata of pdf.

In [ ]:
# Open the PDF document
pdf_doc = fitz.open(f"E:/Model Matiral/the model/toughtimesneverlast.pdf")

num_pages = pdf_doc.page_count
print(f"Number of pages: {num_pages}")
metadata = pdf_doc.metadata
print(f"Metadata: {metadata}")

Convert It To Text

In [ ]:
text_of_book = ""

# Loop through all pages and concatenate text
for page_number in range(pdf_doc.page_count):
    page = pdf_doc[page_number]
    text = page.get_text()
    
    # Concatenate text from each page
    text_of_book += text

# Print the concatenated text. This concatnated text is the whole book.
print(text_of_book)

**Function to summarize text**

In [ ]:
def abstractive_summarization(text, model_path=model_path, max_length= 1000):
    
    # Tokenize and generate summary
    inputs_ids = tokenizer.encode(text, return_tensors="pt",truncation=True,max_length=2000)
    
    # Global attention on the first token 
    global_attention_mask = torch.zeros_like(inputs_ids)
    global_attention_mask[:, 0] = 1
    
    sequences = model.generate(inputs_ids, global_attention_mask=global_attention_mask, max_length=2000, min_length=50,length_penalty=2.0, num_beams=5, early_stopping=True).sequences
    summary = tokenizer.batch_decode(sequences[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)

    return summary

Perform abstractive summarization

In [ ]:
summarized_text = abstractive_summarization(text_of_book)


print("Original Text Length:", len(text_of_book))
print("Summarized Text Length:", len(summarized_text))
print("\nSummarized Text:\n", summarized_text)


In [ ]:
wordcloud = WordCloud(width=800, height=400, background_color='black').generate(text_of_book)

# Display the generated word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()